In [2]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt


In [3]:
df = pd.read_csv('train.csv', encoding=('ISO-8859-1'))

In [4]:
df.shape

(404290, 6)

In [5]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0


In [6]:
new_df = df.sample(30000)

In [7]:
new_df.isnull().sum()

id              0
qid1            0
qid2            0
question1       0
question2       0
is_duplicate    0
dtype: int64

In [8]:
new_df.duplicated().sum()

0

In [9]:
ques_df= new_df[['question1','question2']]
ques_df.head()

,question1,question2
231460,What are some tips to concentrate on studies w...,How do I concentrate while studying?
56850,What is the best way you can get your ex-boyfr...,What are the best ways to get your ex-girlfrie...
396769,"If I want to keep a stock for 2 years, which o...",What are best oil stock to buy right now?
63194,What's the best way to sell my restaurant?,How can I sell my product to a restaurant?
118272,I dreamt I saved a person from drowning. Can i...,I dreamt about 'Shits come out from my mouth'....


In [10]:
from sklearn.feature_extraction.text import CountVectorizer
# merge texts
questions = list(ques_df['question1'].astype(str)) + list(ques_df['question2'].astype(str))

cv = CountVectorizer(max_features=3000)
q1_arr, q2_arr = np.vsplit(cv.fit_transform(questions).toarray(),2)

In [11]:
temp_df1 = pd.DataFrame(q1_arr,index = ques_df.index)
temp_df2 = pd.DataFrame(q2_arr, index = ques_df.index)
temp_df = pd.concat([temp_df1, temp_df2], axis =1)
temp_df.shape

(30000, 6000)

In [12]:
 temp_df

,0,1,2,3,4,5,6,7,8,9,...,2990,2991,2992,2993,2994,2995,2996,2997,2998,2999
231460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56850,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
396769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4916,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
60093,0,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
176509,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
250554,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [13]:
temp_df['is_duplicate']= new_df['is_duplicate']


In [14]:
temp_df.head()

,0,1,2,3,4,5,6,7,8,9,...,2991,2992,2993,2994,2995,2996,2997,2998,2999,is_duplicate
231460,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
56850,0,0,0,0,0,0,0,0,0,0,...,0,1,0,0,0,0,0,0,0,0
396769,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
63194,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
118272,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [15]:
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test = train_test_split(temp_df.iloc[:,0:-1].values,temp_df.iloc[:,-1].values, test_size=0.2, random_state=100)

In [15]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
rf = RandomForestClassifier()
rf.fit(X_train,y_train)
y_pred = rf.predict(X_test)
accuracy_score(y_test,y_pred)

0.7433333333333333

In [17]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb = XGBClassifier()
xgb.fit(X_train,y_train)
y_pred = xgb.predict(X_test)
accuracy_score(y_test,y_pred)

0.726